<a href="https://colab.research.google.com/github/Ananya-AJ/Deep-Learning/blob/main/codelab_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##MNSIT Digits classification

In [1]:
#Datasets path
BATCH_SIZE = 128
EPOCHS = 10

training_images_file   = 'gs://mnist-public/train-images-idx3-ubyte'
training_labels_file   = 'gs://mnist-public/train-labels-idx1-ubyte'
validation_images_file = 'gs://mnist-public/t10k-images-idx3-ubyte'
validation_labels_file = 'gs://mnist-public/t10k-labels-idx1-ubyte'

In [2]:
#imports
import os, re, math, json, shutil, pprint
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import IPython.display as display
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.9.2


In [3]:
#parse files and prepare training and validation datasets
AUTO = tf.data.experimental.AUTOTUNE

def read_label(tf_bytestring):
    label = tf.io.decode_raw(tf_bytestring, tf.uint8)
    label = tf.reshape(label, [])
    label = tf.one_hot(label, 10)
    return label
  
def read_image(tf_bytestring):
    image = tf.io.decode_raw(tf_bytestring, tf.uint8)
    image = tf.cast(image, tf.float32)/256.0
    image = tf.reshape(image, [28*28])
    return image
  
def load_dataset(image_file, label_file):
    imagedataset = tf.data.FixedLengthRecordDataset(image_file, 28*28, header_bytes=16)
    imagedataset = imagedataset.map(read_image, num_parallel_calls=16)
    labelsdataset = tf.data.FixedLengthRecordDataset(label_file, 1, header_bytes=8)
    labelsdataset = labelsdataset.map(read_label, num_parallel_calls=16)
    dataset = tf.data.Dataset.zip((imagedataset, labelsdataset))
    return dataset 
  
def get_training_dataset(image_file, label_file, batch_size):
    dataset = load_dataset(image_file, label_file)
    dataset = dataset.cache()  # this small dataset can be entirely cached in RAM, for TPU this is important to get good performance from such a small dataset
    dataset = dataset.shuffle(5000, reshuffle_each_iteration=True)
    dataset = dataset.repeat() # Mandatory for Keras for now
    dataset = dataset.batch(batch_size, drop_remainder=True) # drop_remainder is important on TPU, batch size must be fixed
    dataset = dataset.prefetch(AUTO)  # fetch next batches while training on the current one (-1: autotune prefetch buffer size)
    return dataset
  
def get_validation_dataset(image_file, label_file):
    dataset = load_dataset(image_file, label_file)
    dataset = dataset.cache() # this small dataset can be entirely cached in RAM, for TPU this is important to get good performance from such a small dataset
    dataset = dataset.batch(10000, drop_remainder=True) # 10000 items in eval dataset, all in one batch
    dataset = dataset.repeat() # Mandatory for Keras for now
    return dataset

In [4]:
# instantiate the datasets
training_dataset = get_training_dataset(training_images_file, training_labels_file, BATCH_SIZE)
validation_dataset = get_validation_dataset(validation_images_file, validation_labels_file)

In [9]:
#Single dense layer keras model
model = tf.keras.Sequential(
  [
      tf.keras.layers.Input(shape=(28*28,)),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 10)                7850      
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [12]:
#model fitting
history = model.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, validation_data=validation_dataset, validation_steps=1)


Epoch 1/10
468/468 [==============================] - 12s 14ms/step - loss: 1.2991 - accuracy: 0.6793 - val_loss: 0.8254 - val_accuracy: 0.8275
Epoch 2/10
468/468 [==============================] - 3s 6ms/step - loss: 0.7233 - accuracy: 0.8358 - val_loss: 0.6134 - val_accuracy: 0.8600
Epoch 3/10
468/468 [==============================] - 3s 6ms/step - loss: 0.5903 - accuracy: 0.8578 - val_loss: 0.5289 - val_accuracy: 0.8743
Epoch 4/10
468/468 [==============================] - 2s 5ms/step - loss: 0.5281 - accuracy: 0.8681 - val_loss: 0.4821 - val_accuracy: 0.8818
Epoch 5/10
468/468 [==============================] - 2s 5ms/step - loss: 0.4887 - accuracy: 0.8756 - val_loss: 0.4516 - val_accuracy: 0.8866
Epoch 6/10
468/468 [==============================] - 3s 5ms/step - loss: 0.4633 - accuracy: 0.8799 - val_loss: 0.4301 - val_accuracy: 0.8899
Epoch 7/10
468/468 [==============================] - 2s 5ms/step - loss: 0.4426 - accuracy: 0.8836 - val_loss: 0.4135 - val_accuracy: 0.8922
Epoc

In [15]:
#predict validation set
probabilities = model.predict(validation_dataset, steps=1)
predicted_labels = np.argmax(probabilities, axis=1)
print(probabilities)


1/1 [==============================] - 2s 2s/step
[[4.6183707e-04 5.3596132e-06 4.1749989e-04 ... 9.9190181e-01
  3.0035275e-04 5.4066749e-03]
 [2.7850334e-02 7.9880044e-04 7.8790081e-01 ... 5.6100776e-06
  1.5689865e-02 2.2484439e-05]
 [6.1581779e-04 9.3526483e-01 1.8712290e-02 ... 6.4876601e-03
  1.4741043e-02 2.7840391e-03]
 ...
 [2.6643567e-05 1.0657802e-04 3.1363167e-04 ... 1.2684549e-02
  2.5582161e-02 7.4805930e-02]
 [1.4677294e-02 3.1935733e-02 4.9440577e-03 ... 7.2448119e-03
  4.3836027e-01 4.1911625e-03]
 [1.2302076e-03 2.4116520e-07 2.4626211e-03 ... 8.9493057e-07
  3.0378407e-05 6.8762697e-06]]


#Adding more layers

In [16]:
model = tf.keras.Sequential(
  [
      tf.keras.layers.Input(shape=(28*28,)),
      tf.keras.layers.Dense(200, activation='sigmoid'),
      tf.keras.layers.Dense(60, activation='sigmoid'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

In [17]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 200)               157000    
                                                                 
 dense_3 (Dense)             (None, 60)                12060     
                                                                 
 dense_4 (Dense)             (None, 10)                610       
                                                                 
Total params: 169,670
Trainable params: 169,670
Non-trainable params: 0
_________________________________________________________________


In [20]:
#train and validate
history = model.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, validation_data=validation_dataset, validation_steps=1)
     

Epoch 1/10
468/468 [==============================] - 3s 6ms/step - loss: 0.7199 - accuracy: 0.8327 - val_loss: 0.3001 - val_accuracy: 0.9188
Epoch 2/10
468/468 [==============================] - 3s 6ms/step - loss: 0.2547 - accuracy: 0.9285 - val_loss: 0.2135 - val_accuracy: 0.9375
Epoch 3/10
468/468 [==============================] - 3s 6ms/step - loss: 0.1893 - accuracy: 0.9452 - val_loss: 0.1672 - val_accuracy: 0.9504
Epoch 4/10
468/468 [==============================] - 3s 6ms/step - loss: 0.1495 - accuracy: 0.9561 - val_loss: 0.1421 - val_accuracy: 0.9579
Epoch 5/10
468/468 [==============================] - 3s 6ms/step - loss: 0.1207 - accuracy: 0.9646 - val_loss: 0.1227 - val_accuracy: 0.9636
Epoch 6/10
468/468 [==============================] - 3s 6ms/step - loss: 0.0988 - accuracy: 0.9711 - val_loss: 0.1047 - val_accuracy: 0.9700
Epoch 7/10
468/468 [==============================] - 4s 8ms/step - loss: 0.0819 - accuracy: 0.9763 - val_loss: 0.0940 - val_accuracy: 0.9712
Epoch 

##Learning rate decay

In [22]:
# lr decay function
def lr_decay(epoch):
  return 0.01 * math.pow(0.6, epoch)

# lr schedule callback
lr_decay_callback = tf.keras.callbacks.LearningRateScheduler(lr_decay, verbose=True)

In [23]:
model_ = tf.keras.Sequential([
      tf.keras.layers.Input(shape=(28*28,)),
      tf.keras.layers.Dense(200, activation='relu'),
      tf.keras.layers.Dense(60, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

# Defining model architecture
model_.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_.summary()

# Fit model
history = model_.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, validation_data=validation_dataset, validation_steps=1, callbacks=[lr_decay_callback])
     

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 200)               157000    
                                                                 
 dense_6 (Dense)             (None, 60)                12060     
                                                                 
 dense_7 (Dense)             (None, 10)                610       
                                                                 
Total params: 169,670
Trainable params: 169,670
Non-trainable params: 0
_________________________________________________________________

Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/10
468/468 [==============================] - 4s 9ms/step - loss: 0.2195 - accuracy: 0.9338 - val_loss: 0.1506 - val_accuracy: 0.9548 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.006.
Epoch 2/10
468/468 [==========

In [24]:
#Adding more layers to the network
model = tf.keras.Sequential([
    tf.keras.layers.Reshape(input_shape=(28*28,), target_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(kernel_size=3, filters=12, activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=6, filters=24, strides=2, activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=6, filters=32, strides=2, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model_.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_.summary()

# Fit model
history = model_.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, validation_data=validation_dataset, validation_steps=1, callbacks=[lr_decay_callback])
     


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 200)               157000    
                                                                 
 dense_6 (Dense)             (None, 60)                12060     
                                                                 
 dense_7 (Dense)             (None, 10)                610       
                                                                 
Total params: 169,670
Trainable params: 169,670
Non-trainable params: 0
_________________________________________________________________

Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/10
468/468 [==============================] - 3s 6ms/step - loss: 0.0989 - accuracy: 0.9710 - val_loss: 0.1208 - val_accuracy: 0.9677 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.006.
Epoch 2/10
468/468 [==========

##Convolutional Neural Network

In [25]:
model = tf.keras.Sequential(
  [
      tf.keras.layers.Reshape(input_shape=(28*28,), target_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(kernel_size=3, filters=12, activation='relu', padding='same'),
      tf.keras.layers.Conv2D(kernel_size=6, filters=24, activation='relu', padding='same', strides=2),
      tf.keras.layers.Conv2D(kernel_size=6, filters=32, activation='relu', padding='same', strides=2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(200, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 12)        120       
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 24)        10392     
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 32)          27680     
                                                                 
 flatten_1 (Flatten)         (None, 1568)              0         
                                                                 
 dense_9 (Dense)             (None, 200)               313800    
                                                                 
 dense_10 (Dense)            (None, 10)               

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [26]:
# Fit model
history = model_.fit(training_dataset, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, validation_data=validation_dataset, validation_steps=1, callbacks=[lr_decay_callback])
     



Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/10
468/468 [==============================] - 3s 6ms/step - loss: 0.2246 - accuracy: 0.9419 - val_loss: 0.1459 - val_accuracy: 0.9645 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.006.
Epoch 2/10
468/468 [==============================] - 3s 6ms/step - loss: 0.0779 - accuracy: 0.9774 - val_loss: 0.1083 - val_accuracy: 0.9737 - lr: 0.0060

Epoch 3: LearningRateScheduler setting learning rate to 0.0036.
Epoch 3/10
468/468 [==============================] - 3s 6ms/step - loss: 0.0434 - accuracy: 0.9869 - val_loss: 0.1012 - val_accuracy: 0.9759 - lr: 0.0036

Epoch 4: LearningRateScheduler setting learning rate to 0.0021599999999999996.
Epoch 4/10
468/468 [==============================] - 4s 8ms/step - loss: 0.0277 - accuracy: 0.9919 - val_loss: 0.1051 - val_accuracy: 0.9755 - lr: 0.0022

Epoch 5: LearningRateScheduler setting learning rate to 0.001296.
Epoch 5/10
468/468 [=====================

##Batch normalisation

In [27]:
model = tf.keras.Sequential(
  [
      tf.keras.layers.Reshape(input_shape=(28*28,), target_shape=(28, 28, 1)),
      
      tf.keras.layers.Conv2D(kernel_size=3, filters=12, use_bias=False, padding='same'),
      tf.keras.layers.BatchNormalization(center=True, scale=False),
      tf.keras.layers.Activation('relu'),
      
      tf.keras.layers.Conv2D(kernel_size=6, filters=24, use_bias=False, padding='same', strides=2),
      tf.keras.layers.BatchNormalization(center=True, scale=False),
      tf.keras.layers.Activation('relu'),
      
      tf.keras.layers.Conv2D(kernel_size=6, filters=32, use_bias=False, padding='same', strides=2),
      tf.keras.layers.BatchNormalization(center=True, scale=False),
      tf.keras.layers.Activation('relu'),
      
      tf.keras.layers.Flatten(),
      
      tf.keras.layers.Dense(200, use_bias=False),
      tf.keras.layers.BatchNormalization(center=True, scale=False),
      tf.keras.layers.Activation('relu'),
      
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 12)        108       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 12)       36        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 28, 28, 12)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 24)        10368     
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 24)       72        
 hNormalization)                                      